In [121]:
from bs4 import BeautifulSoup
        #BeautifulSoup: paquetería de Python que permite leer la estructura HTML para poder ser escrapeada por Python
import urllib3
import pandas as pd
import re

In [139]:
url_to_scrape='http://www.economia-sniim.gob.mx/2010prueba/PreciosHoy.asp?Cons=D&prodC=9002&dest=160&Formato=Xls&submit=Ver+Resultados'
        #Dirección url de la página que realizará la consulta
print(url_to_scrape)

http://www.economia-sniim.gob.mx/2010prueba/PreciosHoy.asp?Cons=D&prodC=9002&dest=160&Formato=Xls&submit=Ver+Resultados


In [140]:
http = urllib3.PoolManager()

response = http.request('GET', url_to_scrape)

soup = BeautifulSoup(response.data)
table=soup.findAll("table")
error=-5

/Users/usuario/anaconda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /Users/usuario/anaconda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [141]:
for i in range(1,len(table)):
        try:
            if table[i]['id'] =='error':
                error=i
        except Exception as e:
            pass
                #print table[i]['id']
  

In [142]:
if error!=-5:
    if table[error].findAll('td')[0].text==u'\nNo encontr\xf3 informaci\xf3n para los datos indicados\n':
        print (u'\nNo encontr\xf3 informaci\xf3n para los datos indicados\n')
else:
    for i in range(1,len(table)):
        try:
            if table[i]['id'] =='Datos':
                indice=i
        except Exception as e:
            pass

In [159]:
df=pd.read_html(str(table[25]))[0]
df.columns = ['origen', 'presentacion', 'pmin', 'pmax', 'pfrec', 'observaciones'] 
df["fecha"] =df["origen"][3]
df1 = df.iloc[6:] 
df1=df1.dropna(thresh=5)
df1.reset_index(drop=True, inplace=True)  

df1

    


,origen,presentacion,pmin,pmax,pfrec,observaciones,fecha
0,Michoacán,Caja de 19 kg.,845.00,855.00,855.00,NaN,Hoy: Miércoles 25/Jul/2018
1,NaN,Caja de 19 kg.,845.00,855.00,855.00,Ayer: 24/07/2018 Ver Nota,Hoy: Miércoles 25/Jul/2018


In [160]:
for i in range(len(df1)):
    if df1["presentacion"][i]== "Kilogramo":
        df1["presentacion"][i]=1
    else:
        df1["presentacion"][i]=re.findall(r'\d+',df1['presentacion'][i])[0]

df1

,origen,presentacion,pmin,pmax,pfrec,observaciones,fecha
0,Michoacán,19,845.00,855.00,855.00,NaN,Hoy: Miércoles 25/Jul/2018
1,NaN,19,845.00,855.00,855.00,Ayer: 24/07/2018 Ver Nota,Hoy: Miércoles 25/Jul/2018


In [161]:
df1["precio_kilo"]=pd.to_numeric(df1["pfrec"])/pd.to_numeric(df1["presentacion"])
df1

,origen,presentacion,pmin,pmax,pfrec,observaciones,fecha,precio_kilo
0,Michoacán,19,845.00,855.00,855.00,NaN,Hoy: Miércoles 25/Jul/2018,45.0
1,NaN,19,845.00,855.00,855.00,Ayer: 24/07/2018 Ver Nota,Hoy: Miércoles 25/Jul/2018,45.0


In [162]:
df1["precio_kilo"].mean()

45.0